<a href="https://colab.research.google.com/github/kjuliek/botdiscord/blob/main/botdiscord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install discord.py
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [30]:
# Structures

# chained list
class node :
  def __init__(self, data, link):
    self.data = data
    self.next_node = link

class chained_list :
  def __init__(self):
    self.first_node = None
    self.last_node = None
    self.current_node = None
    self.length = 0

  def append(self, data):
    self.length += 1
    if self.last_node == None :
      self.first_node = node(data, None)
      self.last_node = self.first_node
      self.current_node = self.first_node
      return
    self.last_node.next_node = node(data, None)
    self.last_node = self.last_node.next_node


  def get(self, id) :
    current_node = self.first_node
    if current_node == None or self.length < id :
      return "erreur"
    i = 0
    while i < id :
      current_node = current_node.next_node
      i += 1
    return current_node.data

  def search(self, data) :
    current_node = self.first_node
    while current_node != None:
      if current_node.data == data :
        return True
      current_node = current_node.next_node
    return False

  def insert(self, data, id) :
    current_node = self.first_node
    if id < 0 or self.length < id :
      return "erreur"
    i = 0
    self.length += 1
    while i < id-1 :
      current_node = current_node.next_node
      i+=1
    next_link = current_node.next_node
    current_node.next_node = node(data, next_link)

  def remove(self, id) :
    current_node = self.first_node
    if current_node == None or self.length-1 < id :
      return "erreur"
    self.length -= 1
    for _ in range(id-1):
      current_node = current_node.next_node
    current_node.next_node = current_node.next_node.next_node

  def remove_all(self):
    self.first_node = None
    self.last_node = None
    self.current_node = None
    self.length = 0

  def previous_node(self):
    current_node = self.first_node
    while current_node.next_node != self.current_node :
      current_node = current_node.next_node
    return current_node

  def To_string_chained_list(self):
    to_send = ""
    current_node = self.first_node
    while current_node != None:
      to_send += To_string(current_node.data)
      current_node = current_node.next_node
    return to_send


# Tree
class Node :
  def __init__(self, data):
    self.data = data
    self.next_nodes = {}

  def add_node(self,question,answer,next_question):
    if self.data == question:
      self.next_nodes[answer] = Node(next_question)
    else :
      for key, value in self.next_nodes.items():
        value.add_node(question,answer,next_question)

  def search(self, data):
    for key, value in self.next_nodes.items():
      if key == data :
        return True
    return False


class Tree:
  def __init__(self):
    self.first_node = None
    self.current_node = None

  def Add_node(self,question,answer,next_question):
    if self.first_node == None:
      self.first_node = Node(question)
    self.first_node.add_node(question,answer,next_question)

  def search(self,data):
    if self.first_node == None:
      return False
    else :
      result = self.first_node.search(data)
      if result == None:
        return False
      else:
        return True

  def evolve(self, answer):
    if self.current_node.search(answer):
      self.current_node = self.current_node.next_nodes[answer]


  def advance_historical(self, ctx, Historical, answer) :

    string_of_members = To_string_members(ctx.guild.members, 3)
    question_0 = "Que voulez-vous faire ? \n 1 : Afficher tout l'historique \n 2 : Afficher la dernière commande \n 3 : Vider l'historique \n 4 : Sortir"
    question_1 = "Voulez-vous voir l'historique d'un utilisateur spécifique ? \n 1 : Tout l'historique \n 2 : " + string_of_members + " Sortir"
    question_2 = "Voulez-vous voir d'autres commandes ? \n 1 : Commande précédente \n 2 : Commande suivante \n 3 : Sortir"

    if self.current_node != None :
      if self.current_node.data == question_1:
        for key in range(len(ctx.guild.members) + 2):
          if answer == str(key+1):
            self.current_node = Node("N/A")
            to_send, historical_remove = answer_reading(ctx, Historical, question_1, answer)
            return to_send, historical_remove
      for key, value in self.current_node.next_nodes.items():
        if self.current_node.data == question_2 and (answer == "1" or answer == "2") :
          to_send, historical_remove = answer_reading(ctx, Historical, question_2, answer)
          return to_send, historical_remove
        elif answer == key :
          question = self.current_node.data
          self.current_node = value
          to_send, historical_remove = answer_reading(ctx, Historical, question, answer)
          return to_send, historical_remove
      return "Je n'ai pas compris votre réponse", False

  def print_tree(self, indentation):
    to_send = ""
    indentation += "    "
    if self.current_node != None :
      to_send += indentation + self.current_node.data + "\n"
      for key, value in self.current_node.next_nodes.items():
        to_send += indentation + key + "\n"
        self.current_node = value
        to_send += self.print_tree(indentation)
    return to_send

# Hashmap
class Hashmap :
  def __init__(self, size):
    self.buckets = []
    for i in range(size):
      self.buckets.append([])

  def add_key_value(self, key, value):
    indice = hash(key) % len(self.buckets)

    L = self.buckets[indice]

    for k in range(len(L)) :
      if L[k][0] == key:
        L[k] = (key, value)
        return
    L.append((key, value))

  def get(self, key) :
    indice = hash(key) % len(self.buckets)

    for (k, value) in self.buckets[indice]:
      if k == key:
        return value

In [31]:
# Historique
def init_historical_tree(guild):
  historical_tree = Tree()


  string_of_members = To_string_members(guild.members, 3)

  question_0 = "Que voulez-vous faire ? \n 1 : Afficher tout l'historique \n 2 : Afficher la dernière commande \n 3 : Vider l'historique \n 4 : Sortir"
  question_1 = "Voulez-vous voir l'historique d'un utilisateur spécifique ? \n 1 : Tout l'historique \n 2 : " + string_of_members + " Sortir"
  question_2 = "Voulez-vous voir d'autres commandes ? \n 1 : Commande précédente \n 2 : Commande suivante \n 3 : Sortir"

  historical_tree.Add_node(question_0, "1", question_1)
  historical_tree.Add_node(question_0, "2", question_2)
  historical_tree.Add_node(question_0, "3", "N/A")
  historical_tree.Add_node(question_0, "4", "N/A")
  historical_tree.Add_node(question_1, "1", "N/A")
  historical_tree.Add_node(question_1, "2", "N/A")
  historical_tree.Add_node(question_1, "3", "N/A")
  historical_tree.Add_node(question_2, "1", "N/A")
  historical_tree.Add_node(question_2, "2", "N/A")
  historical_tree.Add_node(question_2, "3", "N/A")
  historical_tree.current_node = historical_tree.first_node
  return historical_tree

def answer_reading(ctx, Historical, question, answer):
  historical = Historical.get(ctx.guild.name).get(ctx.author.name)
  string_of_members = To_string_members(ctx.guild.members, 3)
  question_0 = "Que voulez-vous faire ? \n 1 : Afficher tout l'historique \n 2 : Afficher la dernière commande \n 3 : Vider l'historique \n 4 : Sortir"
  question_1 = "Voulez-vous voir l'historique d'un utilisateur spécifique ? \n 1 : Tout l'historique \n 2 : " + string_of_members + " Sortir"
  question_2 = "Voulez-vous voir d'autres commandes ? \n 1 : Commande précédente \n 2 : Commande suivante \n 3 : Sortir"

  if question == question_0:
    if answer == "2":
      historical.current_node = historical.last_node
      return To_string(ctx), False
    elif answer == "3":
      return "L'historique à été vidé", True

  elif question == question_1 :
    if answer == "1":
      to_send = To_string_historical(ctx, Historical, "all")
    elif answer != str(len(ctx.guild.members)+2):
      to_send = To_string_historical(ctx, Historical, ctx.guild.members[int(answer)-2].name)
      if to_send == "":
        to_send = "L'historique associé à " + ctx.guild.members[int(answer)-2].name + " est vide"
    return to_send, False

  elif question == question_2:
    if answer == "1":
      historical.current_node = historical.previous_node()
      return To_string(ctx), False
    elif answer == "2":
      if historical.current_node == historical.last_node:
        return "il n'y a pas eu de commande après, la dernière commande était : \n" + To_string(historical.last_node.data), False
      else :
        next_order = historical.current_node.next_node
        historical.current_node = historical.current_node.next_node
        return To_string(ctx), False
  return None, False

def To_string(ctx):
  formatted_time = ctx.message.created_at.strftime("%d-%m-%Y %H:%M:%S")
  return "**" + str(ctx.author) + "** " + formatted_time + " : ```" + str(ctx.command) + "```"

def To_string_members(members, nb):
  to_send = ""
  for member in members:
    if member != client.user:
      to_send += member.name
      to_send +=" \n " + str(nb) + " : "
      nb += 1
  return to_send

def To_string_historical(ctx, Historical, member) :
  to_send = ""
  for m in ctx.guild.members:
    if m.name == member or member == "all":
      to_send += Historical.get(ctx.guild.name).get(m.name).To_string_chained_list()
  return to_send


In [77]:
import random

# Help
def init_discussion_tree() :
  question_0 = "Choisissez une catégorie :"
  question_1 =  "Choisissez une sous-catégorie de Culture et Connaissance :"
  question_2 =  "Choisissez une sous-catégorie de Divertissement :"
  question_3 =  "Choisissez une sous-catégorie de Science et Nature :"
  question_4 =  "Faites un choix dans Géographie :"
  question_5 =  "Faites un choix :"
  question_6 =  "Faites un choix dans Informatique :"
  question_7 =  "Choisissez une difficulté :"
  question_8 =  "Faites un choix dans ce langage :"

  discussion_tree = Tree()

  discussion_tree.Add_node(question_0, "Culture et Connaissance", question_1)
  discussion_tree.Add_node(question_0, "Divertissement", question_2)
  discussion_tree.Add_node(question_0, "Science et Nature", question_3)
  discussion_tree.Add_node(question_0, "Quitter", "N/A")

  discussion_tree.Add_node(question_1, "Géographie", question_4)
  discussion_tree.Add_node(question_1, "Histoire", question_5)
  discussion_tree.Add_node(question_1, "Culture Générale", question_5)
  discussion_tree.Add_node(question_1, "Célébrités", question_5)
  discussion_tree.Add_node(question_1, "Mythologie", question_5)
  discussion_tree.Add_node(question_1, "Politique", question_5)
  discussion_tree.Add_node(question_1, "Quitter", "N/A")

  discussion_tree.Add_node(question_2, "Art", question_5)
  discussion_tree.Add_node(question_2, "Blague", "N/A")
  discussion_tree.Add_node(question_2, "Sports", question_5)
  discussion_tree.Add_node(question_2, "Gadgets", question_5)
  discussion_tree.Add_node(question_2, "Véhicules", question_5)
  discussion_tree.Add_node(question_2, "Quitter", "N/A")

  discussion_tree.Add_node(question_3, "Connaissance générale", question_5)
  discussion_tree.Add_node(question_3, "Animaux", question_5)
  discussion_tree.Add_node(question_3, "Mathématiques", question_5)
  discussion_tree.Add_node(question_3, "Informatique", question_6)
  discussion_tree.Add_node(question_3, "Quitter", "N/A")

  discussion_tree.Add_node(question_4, "Pays Aléatoire", "N/A")
  discussion_tree.Add_node(question_4, "Quiz", question_7)
  discussion_tree.Add_node(question_4, "Article", "N/A")
  discussion_tree.Add_node(question_4, "Quitter", "N/A")

  discussion_tree.Add_node(question_5, "Quiz", question_7)
  discussion_tree.Add_node(question_5, "Article", "N/A")
  discussion_tree.Add_node(question_5, "Quitter", "N/A")

  discussion_tree.Add_node(question_6, "C#", question_8)
  discussion_tree.Add_node(question_6, "Go", question_8)
  discussion_tree.Add_node(question_6, "Java", question_8)
  discussion_tree.Add_node(question_6, "JavaScript", question_8)
  discussion_tree.Add_node(question_6, "Python", question_8)
  discussion_tree.Add_node(question_6, "SQLite", question_8)
  discussion_tree.Add_node(question_6, "Article", question_8)
  discussion_tree.Add_node(question_6, "Quitter", "N/A")

  discussion_tree.Add_node(question_7, "Facile", "N/A")
  discussion_tree.Add_node(question_7, "Moyen", "N/A")
  discussion_tree.Add_node(question_7, "Difficile", "N/A")
  discussion_tree.Add_node(question_7, "Quitter", "N/A")

  discussion_tree.Add_node(question_8, "Documentation", "N/A")
  discussion_tree.Add_node(question_8, "Article", "N/A")
  discussion_tree.Add_node(question_8, "Quitter", "N/A")

  discussion_tree.current_node = discussion_tree.first_node
  return discussion_tree

def answer_reading_discussion(ctx, answer):
  if answer == "Blague" :
    Discussion_tree.get(ctx.guild.name).get(ctx.author.name).evolve(answer)
    return joke()
  elif answer == "Pays Aléatoire" :
    return  get_random_country()
  elif answer == "Facile" :
    return
  elif answer == "Moyen" :
    return
  elif answer == "Difficile" :
    return
  elif answer == "Article" :
    return
  elif answer == "Documentation":
    Discussion_tree.get(ctx.guild.name).get(ctx.author.name).evolve(answer)
    return
  else :
    Discussion_tree.get(ctx.guild.name).get(ctx.author.name).evolve(answer)
    return

def joke():
  try :
    response = requests.get("https://api.chucknorris.io/jokes/random")
    data = response.json()
    if response.status_code == 200:
      return data["value"]
    else:
      return f"Erreur de l'API Chuck Norris Jokes : {data.get('error', 'Erreur inconnue')}"
  except Exception as e:
    return f"Une erreur s'est produite : {e}"

def get_random_country():
  try:
    response = requests.get("https://restcountries.com/v3.1/all")
    data = response.json()

    if response.status_code == 200:
      list_size = len(data)
      random_index = random.randint(0, list_size - 1)
      random_country = data[random_index]
      name = random_country[0]['translations']['fra']['common']
      print(name)
      continent = random_country[0]['region']
      capital = random_country[0]['capital']
      to_send = name + "\n Continent : " + continent + "\n Capitale :" + capital
      return to_send


    else:
      return f"Erreur de l'API Rest Countries : {data.get('error', 'Erreur inconnue')}"
  except Exception as e:
      return f"Une erreur s'est produite : {e}"





In [33]:
# Buttons

#question_0
class Button_category(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="Culture et Connaissance", style=discord.ButtonStyle.primary)
  async def culture_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Culture et Connaissance"
    await interaction.response.send_message("Vous avez choisi la catégorie Culture et Connaissance.",ephemeral=True)

  @discord.ui.button(label="Divertissement", style=discord.ButtonStyle.danger)
  async def entertainment_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Divertissement"
    await interaction.response.send_message("Vous avez choisi la catégorie Divertissement",ephemeral=True)

  @discord.ui.button(label="Science et Nature", style=discord.ButtonStyle.success)
  async def science_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Science et Nature"
    await interaction.response.send_message("Vous avez choisi la catégorie Science et Nature",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer

#question_1
class Button_category_culture(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="Géographie", style=discord.ButtonStyle.primary)
  async def geographie_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Géographie"
    await interaction.response.send_message("Vous avez choisi la catégorie Géographie",ephemeral=True)

  @discord.ui.button(label="Histoire", style=discord.ButtonStyle.primary)
  async def history_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Histoire"
    await interaction.response.send_message("Vous avez choisi la catégorie Histoire",ephemeral=True)

  @discord.ui.button(label="Culture Générale", style=discord.ButtonStyle.primary)
  async def general_knowledge_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Culture Générale"
    await interaction.response.send_message("Vous avez choisi la catégorie Culture Générale",ephemeral=True)

  @discord.ui.button(label="Célébrités", style=discord.ButtonStyle.primary)
  async def celebrities_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Célébrités"
    await interaction.response.send_message("Vous avez choisi la catégorie Célébrités",ephemeral=True)

  @discord.ui.button(label="Mythologie", style=discord.ButtonStyle.primary)
  async def mythology_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Mythologie"
    await interaction.response.send_message("Vous avez choisi la catégorie Mythologie",ephemeral=True)

  @discord.ui.button(label="Politique", style=discord.ButtonStyle.primary)
  async def politic_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Politique"
    await interaction.response.send_message("Vous avez choisi la catégorie Politique",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer

#question_2
class Button_category_entertainment(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="Art", style=discord.ButtonStyle.danger)
  async def art_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Art"
    await interaction.response.send_message("Vous avez choisi Art",ephemeral=True)

  @discord.ui.button(label="Blague", style=discord.ButtonStyle.danger)
  async def blague_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Blague"
    await interaction.response.send_message("Vous avez choisi Blague",ephemeral=True)

  @discord.ui.button(label="Sports", style=discord.ButtonStyle.danger)
  async def sport_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Sports"
    await interaction.response.send_message("Vous avez choisi Sports",ephemeral=True)

  @discord.ui.button(label="Gadgets", style=discord.ButtonStyle.danger)
  async def gadgets_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Gadgets"
    await interaction.response.send_message("Vous avez choisi Gadgets",ephemeral=True)

  @discord.ui.button(label="Vehicules", style=discord.ButtonStyle.danger)
  async def vehicules_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Vehicules"
    await interaction.response.send_message("Vous avez choisi Vehicules",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer

#question_3
class Button_category_science_and_nature(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="Connaissance Générales", style=discord.ButtonStyle.success)
  async def general_knowledge_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Connaissance Générales"
    await interaction.response.send_message("Vous avez choisi Connaissance Générales",ephemeral=True)

  @discord.ui.button(label="Animaux", style=discord.ButtonStyle.success)
  async def animals_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Animaux"
    await interaction.response.send_message("Vous avez choisi Animaux",ephemeral=True)

  @discord.ui.button(label="Mathématiques", style=discord.ButtonStyle.success)
  async def maths_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Mathématiques"
    await interaction.response.send_message("Vous avez choisi Mathématiques",ephemeral=True)

  @discord.ui.button(label="Informatique", style=discord.ButtonStyle.success)
  async def informatique_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Informatique"
    await interaction.response.send_message("Vous avez choisi Informatique",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer

#question_4
class Button_geography(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="Pays Aléatoire", style=discord.ButtonStyle.primary)
  async def random_country_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Pays Aléatoire"
    await interaction.response.send_message("Vous avez choisi Pays Aléatoire",ephemeral=True)

  @discord.ui.button(label="Quiz", style=discord.ButtonStyle.secondary)
  async def quiz_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quiz"
    await interaction.response.send_message("Vous avez choisi Quiz",ephemeral=True)

  @discord.ui.button(label="Article", style=discord.ButtonStyle.secondary)
  async def article_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Article"
    await interaction.response.send_message("Vous avez choisi Article",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer

#question_5
class Button_quiz(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="Quiz", style=discord.ButtonStyle.secondary)
  async def quiz_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quiz"
    await interaction.response.send_message("Vous avez choisi Quiz",ephemeral=True)

  @discord.ui.button(label="Article", style=discord.ButtonStyle.secondary)
  async def article_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Article"
    await interaction.response.send_message("Vous avez choisi Article",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer

#question_6
class Button_language(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="C#", style=discord.ButtonStyle.success)
  async def c_sharp_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "C#"
    await interaction.response.send_message("Vous avez choisi C#",ephemeral=True)

  @discord.ui.button(label="Go", style=discord.ButtonStyle.success)
  async def go_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Go"
    await interaction.response.send_message("Vous avez choisi Go",ephemeral=True)

  @discord.ui.button(label="Java", style=discord.ButtonStyle.success)
  async def java_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Java"
    await interaction.response.send_message("Vous avez choisi Java",ephemeral=True)

  @discord.ui.button(label="JavaScript", style=discord.ButtonStyle.success)
  async def js_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "JavaScript"
    await interaction.response.send_message("Vous avez choisi JavaScript",ephemeral=True)

  @discord.ui.button(label="Python", style=discord.ButtonStyle.success)
  async def python_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Python"
    await interaction.response.send_message("Vous avez choisi Python",ephemeral=True)

  @discord.ui.button(label="SQLite", style=discord.ButtonStyle.success)
  async def sql_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "SQLite"
    await interaction.response.send_message("Vous avez choisi SQLite",ephemeral=True)

  @discord.ui.button(label="Article", style=discord.ButtonStyle.secondary)
  async def article_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Article"
    await interaction.response.send_message("Vous avez choisi Article",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer

#question_7
class Button_difficulty(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="Facile", style=discord.ButtonStyle.success)
  async def easy_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Facile"
    await interaction.response.send_message("Vous avez choisi Facile",ephemeral=True)

  @discord.ui.button(label="Moyen", style=discord.ButtonStyle.secondary)
  async def medium_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Moyen"
    await interaction.response.send_message("Vous avez choisi Moyen",ephemeral=True)

  @discord.ui.button(label="Difficile", style=discord.ButtonStyle.danger)
  async def hard_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Difficile"
    await interaction.response.send_message("Vous avez choisi Difficile",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer

#question_8
class Button_documentation(discord.ui.View):
  def __init__(self):
    super().__init__()
    Button_category.click = False
    Button_category.answer = ""

  @discord.ui.button(label="Documentation", style=discord.ButtonStyle.success)
  async def documentation_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Documentation"
    await interaction.response.send_message("Vous avez choisi Documentation",ephemeral=True)

  @discord.ui.button(label="Article", style=discord.ButtonStyle.secondary)
  async def article_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Article"
    await interaction.response.send_message("Vous avez choisi Article",ephemeral=True)

  @discord.ui.button(label="Quitter", style=discord.ButtonStyle.secondary)
  async def exit_button(self, interaction: discord.Interaction , button : discord.ui.Button):
    Button_category.click = True
    Button_category.answer = "Quitter"
    await interaction.response.send_message("Vous avez choisi de Quitter",ephemeral=True)

  def Click(self):
    return Button_category.click

  def Answer(self):
    return Button_category.answer


def View(question):
  question_0 = "Choisissez une catégorie :"
  question_1 = "Choisissez une sous-catégorie de Culture et Connaissance :"
  question_2 = "Choisissez une sous-catégorie de Divertissement :"
  question_3 = "Choisissez une sous-catégorie de Science et Nature :"
  question_4 = "Faites un choix dans Géographie :"
  question_5 = "Faites un choix :"
  question_6 = "Faites un choix dans Informatique :"
  question_7 = "Choisissez une difficulté :"
  question_8 = "Faites un choix dans ce langage :"

  if question == question_0:
    return Button_category()
  elif question == question_1:
    return Button_category_culture()
  elif question == question_2:
    return Button_category_entertainment()
  elif question == question_3:
    return Button_category_science_and_nature()
  elif question == question_4:
    return Button_geography()
  elif question == question_5:
    return Button_quiz()
  elif question == question_6:
    return Button_language()
  elif question == question_7:
    return Button_difficulty()
  elif question == question_8:
    return Button_documentation()



In [ ]:
import discord
import asyncio
import requests

intents = discord.Intents.all()

from discord.ext import commands

client = commands.Bot(command_prefix="!", intents = intents)

Historical = Hashmap(10)
Historical_tree = Hashmap(10)
Discussion_tree = Hashmap(10)

@client.command(name="delete")
async def delete(ctx):
    Historical.get(ctx.guild.name).get(ctx.author.name).append(ctx)
    messages = []
    async for message in ctx.channel.history(limit=10):
        messages.append(message)

    for each_message in messages:
        await each_message.delete()

@client.command(name="capitale")
async def capitale(ctx, pays: str):
  Historical.get(ctx.guild.name).get(ctx.author.name).append(ctx)
  try:
    response = requests.get(f"https://restcountries.com/v3.1/translation/{pays}")
    data = response.json()

    if response.status_code == 200:
      capital = data[0]['capital'][0]
      pays = data[0]['translations']['fra']['common']
      await ctx.send(f"La capitale de {pays} est {capital}.")
    else:
      await ctx.send(f"Erreur : Impossible d'obtenir des informations sur {pays}.")

  except Exception as e:
    await ctx.send(f"Une erreur s'est produite : {e}")

@client.command(name="blague")
async def blague(ctx):
  Historical.get(ctx.guild.name).get(ctx.author.name).append(ctx)
  await ctx.send(joke())

@client.command(name="pays_aleatoire")
async def pays_aleatoire(ctx):
  Historical.get(ctx.guild.name).get(ctx.author.name).append(ctx)
  to_send = get_random_country()
  if to_send != None:
      await ctx.send(to_send)


@client.command(name="helpp")
async def helpp(ctx):
  try :
    Historical.get(ctx.guild.name).get(ctx.author.name).append(ctx)
    while Discussion_tree.get(ctx.guild.name).get(ctx.author.name).current_node.data != "N/A":
      question = Discussion_tree.get(ctx.guild.name).get(ctx.author.name).current_node.data
      view = View(question)
      await ctx.channel.send(question, view=View(question))
      waiting = 0
      while not view.Click() and waiting < 60 and not client.is_closed():
        waiting += 1
        await asyncio.sleep(1)
      if not(view.Click()):
          if not client.is_closed():
              await ctx.send('Temps écoulé. Aucun message reçu.')
              break  # Sortir de la boucle en cas de timeout
      to_send = answer_reading_discussion(ctx, view.Answer())
      if to_send != None:
          await ctx.send(to_send)
    Discussion_tree.get(ctx.guild.name).get(ctx.author.name).current_node = Discussion_tree.get(ctx.guild.name).get(ctx.author.name).first_node


  except Exception as e:
        await ctx.send(f"Une erreur s'est produite : {e}")

@client.command(name="historique")
async def historique(ctx):
    try:
        Historical.get(ctx.guild.name).get(ctx.author.name).append(ctx)
        while Historical_tree.get(ctx.guild.name).get(ctx.author.name).current_node.data != "N/A":
            question = Historical_tree.get(ctx.guild.name).get(ctx.author.name).current_node.data
            await ctx.channel.send(question)
            try:
                message = await client.wait_for('message', check=lambda m: m.channel == ctx.channel, timeout=60.0)
                to_send, remove_historical = Historical_tree.get(ctx.guild.name).get(ctx.author.name).advance_historical(ctx, Historical, message.content)
            except asyncio.TimeoutError:
                if not client.is_closed():
                    await ctx.send('Temps écoulé. Aucun message reçu.')
                    break  # Sortir de la boucle en cas de timeout
            if to_send is not None :
              await ctx.send(to_send)
            if remove_historical :
                Historical.get(ctx.guild.name).get(ctx.author.name).remove_all()
        Historical_tree.get(ctx.guild.name).get(ctx.author.name).current_node = Historical_tree.get(ctx.guild.name).get(ctx.author.name).first_node
    except Exception as e:
        if not client.is_closed():
            await ctx.send(f"Une erreur s'est produite : {e}")

@client.event
async def on_ready(): #quand en ligne
  print("Le bot est prêt !")

  for guild in client.guilds:
    h = Hashmap(100)
    h_t = Hashmap(100)
    d_t = Hashmap(100)
    for member in guild.members:
        h.add_key_value(member.name, chained_list())
        h_t.add_key_value(member.name, init_historical_tree(guild))
        d_t.add_key_value(member.name, init_discussion_tree())
    Historical.add_key_value(guild.name, h)
    Historical_tree.add_key_value(guild.name, h_t)
    Discussion_tree.add_key_value(guild.name, d_t)

@client.event
async def on_member_join(member):
    general_channel = client.get_channel(1044900412551073832)
    if member.name is not None:
      await general_channel.send("Bienvenue sur le serveur ! "+ member.name)

client.run("key")